In [7]:
# Cell 1: Imports and Setup
# All necessary imports and API key setup
import os
import yaml
import anthropic
import time
import random
from datetime import datetime
from pathlib import Path
import re

print(f"Anthropic version: {anthropic.__version__}")
os.environ['ANTHROPIC_API_KEY'] = "your-api-here"  # Replace with your actual API key

# Directory paths
INPUT_DIR = "/Users/kemi/Documents/GitHub/vocab/src/content/articles"
OUTPUT_DIR = "/Users/kemi/Documents/GitHub/vocab/src/components/articles"

# Constants for Claude API
MODEL_NAME = "claude-3-5-sonnet-20241022"
MAX_TOKENS = 6000
TEMPERATURE = 0.7

# Print setup confirmation
print("🔧 Environment setup complete")
print(f"📁 Input directory: {INPUT_DIR}")
print(f"📁 Output directory: {OUTPUT_DIR}")

Anthropic version: 0.37.1
🔧 Environment setup complete
📁 Input directory: /Users/kemi/Documents/GitHub/vocab/src/content/articles
📁 Output directory: /Users/kemi/Documents/GitHub/vocab/src/components/articles


In [8]:
# Cell 2 - Base Functions

def create_prompt(title, summary):
    """
    Creates a detailed prompt for the AI to generate a React component.
    
    Args:
        title (str): The concept title
        summary (str): Brief summary of the concept
        
    Returns:
        str: Formatted prompt for Claude
    """
    prompt = f'''
Create an intuitive React component that teaches {title} through visual metaphors and real-world examples.

CONCEPT BREAKDOWN:
1. Name of the concept: {title}
2. Core Principle: {summary}

EDUCATIONAL GOALS:
1. Help the human student understand {title} by connecting it to familiar concepts
2. Show practical applications they might encounter in daily life
3. Build understanding through progressive revelation, not all at once

VISUALIZATION APPROACH:
1. Core Metaphor:
   - Choose 2 or 3 central, relatable metaphors that captures the essence of {title}
   - Example: For "Neural Networks" → use a "Learning to Ride a Bike" metaphor, "Learning to Cook" metaphor or "Learning a New Language" metaphor.
   - Create new metaphor examples
   
2. Animation & Interaction:
   - Start with an automatic demo cycle using useEffect with proper cleanup
   - Implement timing logic using useEffect hook with cleanup functions
   - Allow human user interaction when relevant
   - The interactions might include, but not limited to, moving objects, selecting objects given a criteria, sliding objetcts, Scroll or Pinch-to-zoom, Swipe navigation, Drag and Drop Operations, Scroll-Based Interactions, Interactive tutorials
   - Avoid using just a "next/start/pause/play" button as interaction
   - Provide reset capability
   - Use humor and relatable situations when appropriate
   - Show cause-and-effect clearly

3. Visual Elements:
   - Every visual element should map to a real concept
   - Use Lucide icons as meaningful symbols, not decoration

Remember: A successful visualization is one where users can explain the concept to others after using it.
'''
    return prompt

def extract_frontmatter(content):
    """
    Extract YAML frontmatter from markdown content.
    
    Args:
        content (str): Full markdown file content
        
    Returns:
        dict or None: Extracted frontmatter as dictionary, None if extraction fails
    """
    if content.startswith('---'):
        parts = content.split('---', 2)[1:]
        if len(parts) >= 1:
            try:
                return yaml.safe_load(parts[0])
            except yaml.YAMLError as e:
                print(f"  ⚠️ Error parsing YAML frontmatter: {str(e)}")
                return None
    return None

def format_time(seconds):
    """
    Format seconds into minutes and seconds.
    
    Args:
        seconds (float): Number of seconds
        
    Returns:
        str: Formatted string like "1m 30s"
    """
    return f"{int(seconds // 60)}m {int(seconds % 60)}s"

def get_existing_component_names(output_dir):
    """
    Get names of existing components in the output directory.
    
    Args:
        output_dir (str): Path to components directory
        
    Returns:
        set: Set of component names (without .tsx extension)
    """
    existing_names = set()
    if os.path.exists(output_dir):
        for file in os.listdir(output_dir):
            if file.endswith('.tsx'):
                # Store exact filename without .tsx extension
                existing_names.add(file[:-4])
    return existing_names

# Print confirmation
print("✅ Base functions loaded")

✅ Base functions loaded


In [9]:
# Cell 3 - Component Generation Core

def generate_component(client, prompt):
    """
    Generate React component using Claude API.
    
    Args:
        client: Anthropic client instance
        prompt (str): Generated prompt for component creation
        
    Returns:
        tuple: (component_code, validation_issues)
    """
    system_prompt = '''        
You are a creative expert React developer and AI professor specializing in educational components for 15 to 18-year-old humans. 
Your components must strictly follow these technical requirements:

1. Architecture:
- "use client" directive at start (first line)
- import { useState, useEffect } from "react"; as the second line
- Only useState and useEffect hooks
- Only Lucide icons for visuals
- Only Tailwind CSS for styling
- No external libraries/components
- File extension: .tsx

2. TypeScript Implementation:
interface ComponentProps {
    // Define if needed, empty interface required
}

// All state must use explicit types
const [state, setState] = useState<StateType>(initialValue);

// Event handlers must be typed
const handleEvent = (e: React.MouseEvent<HTMLButtonElement>) => {...};

// Constants outside component
const SCENARIOS: ScenarioType[] = [...];

3. Effects & Cleanup:
useEffect(() => {
    // Effect logic
    return () => {
    // Cleanup required
    };
}, [dependencies]);

4. Styling Standards:
- Only core Tailwind classes
- No arbitrary values (e.g., h-[500px])
- Transitions: duration-300 to duration-500
- Color scheme:
    • Blue (#3B82F6) - active/focus
    • Gray (#6B7280) - background
    • Green (#22C55E) - success

5. Code Organization:
- Max 200 lines per component
- Early returns with type guards
- JSDoc component documentation
- Proper hooks cleanup
- No inline styles
- No setTimeout/setInterval (use useEffect)

Return only raw TSX code without explanations or markdown.
'''
    # Print system prompt and user prompt
    print("\n=== System Prompt ===")
    print(system_prompt)
    print("\n=== User Prompt ===")
    print(prompt)
    print("\n===================")
    
    try:
        response = client.messages.create(
            model=MODEL_NAME,
            max_tokens=MAX_TOKENS,
            temperature=TEMPERATURE,
            system=system_prompt,
            messages=[{"role": "user", "content": prompt}]
        )
        
        component_code = response.content[0].text
        
        # Clean up the code if it's wrapped in markdown
        if component_code.startswith('```'):
            first_newline = component_code.find('\n')
            if first_newline != -1:
                component_code = component_code[first_newline + 1:]
            if component_code.strip().endswith('```'):
                component_code = component_code.strip()[:-3]
        
        # Validate the generated code
        issues = validate_component(component_code)
        
        return component_code, issues
        
    except Exception as e:
        print(f"  ❌ Error generating component: {str(e)}")
        return None, ["Generation failed"]

def generate_component_with_refinement(client, title, prompt):
    """
    Two-stage component generation with iterative refinement.
    
    Args:
        client: Anthropic client instance
        title (str): Component title for logging
        prompt (str): Initial generation prompt
    
    Returns:
        tuple: (final_component_code, final_issues)
    """
    print(f"\n  ⌛ Stage 1: Generating initial component for {title}...")
    
    # Stage 1: Initial Generation
    component_code, issues = generate_component(client, prompt)
    
    if not issues:
        print("  ✅ Initial component passed validation")
        return component_code, issues
        
    # Stage 2: Refinement
    print("\n  ⌛ Stage 2: Refining component...")
    
    # Group issues by category for more structured feedback
    typescript_issues = [i for i in issues if 'TypeScript' in i or 'type' in i.lower()]
    react_issues = [i for i in issues if 'React' in i or 'component' in i.lower()]
    pattern_issues = [i for i in issues if 'pattern' in i.lower() or 'usage' in i]
    
    refinement_system_prompt = """
You are a TypeScript and React expert focusing on fixing technical issues in React components.
Your task is to fix the provided issues while:
1. Preserving the original component's functionality and visual design
2. Maintaining proper TypeScript types and React patterns
3. Ensuring all fixes follow React best practices
4. Not introducing new issues or anti-patterns

Return only the complete fixed component code without any explanations or markdown formatting.
"""

    refinement_user_prompt = f"""
Please fix the following issues in this React component while preserving its core functionality and visual design:

COMPONENT CODE:
```tsx
{component_code}
```

ISSUES TO FIX:

{f'TypeScript Issues:' if typescript_issues else ''}
{chr(10).join(f"- {issue}" for issue in typescript_issues)}

{f'React Issues:' if react_issues else ''}
{chr(10).join(f"- {issue}" for issue in react_issues)}

{f'Pattern Issues:' if pattern_issues else ''}
{chr(10).join(f"- {issue}" for issue in pattern_issues)}

Return only the complete fixed component code without explanations.
Ensure all functionality remains the same while fixing these technical issues.
"""

    # Print refinement prompts
    print("\n=== Refinement System Prompt ===")
    print(refinement_system_prompt)
    print("\n=== Refinement User Prompt ===")
    print(refinement_user_prompt)
    print("\n==============================")
    
    try:
        # Generate refined version with both system and user prompts
        response = client.messages.create(
            model=MODEL_NAME,
            max_tokens=MAX_TOKENS,
            temperature=TEMPERATURE,
            system=refinement_system_prompt,  # Now properly using the system prompt
            messages=[{"role": "user", "content": refinement_user_prompt}]  # Using the proper user prompt
        )
        
        refined_code = response.content[0].text
        refined_issues = validate_component(refined_code)
        
        # Log refinement results
        if len(refined_issues) < len(issues):
            print(f"  ✅ Refinement fixed {len(issues) - len(refined_issues)} issues")
            return refined_code, refined_issues
        else:
            print("  ⚠️ Refinement did not improve component quality")
            return component_code, issues
            
    except Exception as e:
        print(f"  ❌ Error during refinement: {str(e)}")
        return component_code, issues

# Print confirmation
print("✅ Component generation functions loaded")

✅ Component generation functions loaded


In [10]:
# Cell 4 - Component Validation and Fixes

def validate_component(component_code):
    """
    Validate the generated component against best practices and requirements.
    
    Args:
        component_code (str): The generated component code to validate
        
    Returns:
        list: List of validation issues found
    """
    issues = []
    
    # Required TypeScript patterns
    typescript_patterns = {
        'interface': 'No TypeScript interfaces defined',
        'React.FC': 'Missing React.FC type declaration',
        'useState<': 'Missing type parameters in useState',
        'LucideIcon': 'Missing LucideIcon type import/usage',
    }
    
    # Required React patterns
    react_patterns = {
        '"use client";': 'Missing "use client" directive at start',
        'import { useState, useEffect } from "react"': 'Missing or incorrect React imports',
        'export default': 'Missing default export',
        'transition': 'No transitions found for animations',
        'className': 'No Tailwind classes found',
        'onClick': 'No interactive elements found',
        'return': 'No return statement found'
    }
    
    # Anti-patterns to check
    anti_patterns = {
        r'className="[^"]*\[[^\]]*\]"': 'Found arbitrary values in Tailwind classes',
        'setTimeout': 'Direct setTimeout usage found - use useEffect instead',
        'setInterval': 'Direct setInterval usage found - use useEffect instead',
        'style=\\{\\{': 'Inline styles found instead of Tailwind classes',
        'styled': 'Styled-components usage found',
        '@keyframes': 'Direct CSS keyframes found',
        'framer': 'Framer Motion import found',
        r'motion\.': 'Framer Motion component found',
        '@emotion': 'Emotion styling found',
        'any': 'Avoid using the "any" type',
        r'scenarios\[step\]\.icon': 'Accessing dynamic icon properties directly - use type-safe approach',
        r'<scenarios\.': 'Invalid JSX usage of dynamic components',
        r'function\s+\w+\s*\(\s*\)\s*\{': 'Function missing type declaration'
    }
    
    # Check for required patterns
    for pattern, message in {**typescript_patterns, **react_patterns}.items():
        if pattern not in component_code:
            issues.append(f"❌ {message}")
    
    # Check for anti-patterns using regex
    for pattern, message in anti_patterns.items():
        try:
            if re.search(pattern, component_code, re.MULTILINE):
                issues.append(f"❌ {message}")
        except re.error as e:
            print(f"Warning: Invalid regex pattern '{pattern}': {str(e)}")
            continue
    
    # Check number of hooks
    if component_code.count('useState') > 5:
        issues.append("⚠️ Too many useState hooks (>5) - consider combining related state")
    
    if component_code.count('useEffect') > 3:
        issues.append("⚠️ Too many useEffect hooks (>3) - consider combining effects")
    
    # Check component length
    if len(component_code.split('\n')) > 200:
        issues.append("⚠️ Component is too long (>200 lines) - consider breaking it down")
    
    # Check for useEffect cleanup
    try:
        effect_blocks = re.findall(r'useEffect\(\s*\(\s*\)\s*=>\s*\{[^}]*\}', component_code, re.MULTILINE | re.DOTALL)
        for block in effect_blocks:
            if 'return' not in block:
                issues.append("❌ useEffect missing cleanup function")
    except re.error as e:
        print(f"Warning: Error checking useEffect cleanup: {str(e)}")
    
    return issues

def fix_common_typescript_issues(component_code):
    """
    Fix common TypeScript issues in the generated component.
    
    Args:
        component_code (str): Component code to fix
        
    Returns:
        str: Fixed component code
    """
    fixes = []
    
    # Add missing imports
    if 'LucideIcon' not in component_code:
        fixes.append((
            'import { Brain } from "lucide-react";',
            'import { Brain, LucideIcon } from "lucide-react";'
        ))
    
    # Fix dynamic icon usage
    if re.search(r'<\w+\.icon', component_code):
        fixes.append((
            r'<(\w+)\.icon',
            r'{\\1.icon && <\\1.icon'
        ))
    
    # Add missing type definitions
    if not re.search(r'interface Props', component_code):
        fixes.append((
            'export default function',
            'interface Props {}\n\nexport default function'
        ))
    
    # Fix scenarios array type safety
    scenarios_pattern = r'const scenarios = \['
    if re.search(scenarios_pattern, component_code):
        fixes.append((
            scenarios_pattern,
            'const SCENARIOS: Array<ScenarioType> = ['
        ))
    
    # Apply all fixes
    fixed_code = component_code
    for old, new in fixes:
        fixed_code = re.sub(old, new, fixed_code)
    
    # Ensure "use client" directive is properly formatted
    if fixed_code.startswith('use client;'):
        fixed_code = '"use client";' + fixed_code[len('use client;'):]
    elif not fixed_code.startswith('"use client";'):
        fixed_code = '"use client";\n\n' + fixed_code
    
    return fixed_code

def save_tsx_file(content, md_filename, output_dir):
    """
    Save the API response as a .tsx file with TypeScript validation and fixes.
    
    Args:
        content (str): The component code to save
        md_filename (str): Original markdown filename with .md extension
        output_dir (str): Directory to save the TSX file
        
    Returns:
        list: Any validation issues found
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Convert .md to .tsx while preserving exact filename
    tsx_filename = md_filename.replace('.md', '.tsx')
    filepath = os.path.join(output_dir, tsx_filename)
    
    # Clean the content
    cleaned_content = content
    if content.startswith('```'):
        first_newline = content.find('\n')
        if first_newline != -1:
            content = content[first_newline + 1:]
        if content.strip().endswith('```'):
            content = content.strip()[:-3]
    
    # Ensure proper formatting
    cleaned_content = content.strip()
    if cleaned_content.startswith('use client;'):
        cleaned_content = '"use client";' + cleaned_content[len('use client;'):]
    elif not cleaned_content.startswith('"use client";'):
        cleaned_content = '"use client";\n\n' + cleaned_content
    
    # Validate the component
    issues = validate_component(cleaned_content)
    
    # If there are TypeScript-specific issues, try to fix them
    if any('TypeScript' in issue or 'type' in issue.lower() for issue in issues):
        print("  🔧 Attempting to fix TypeScript issues...")
        cleaned_content = fix_common_typescript_issues(cleaned_content)
        # Revalidate after fixes
        issues = validate_component(cleaned_content)
    
    # Print validation results
    if issues:
        print("\n  ⚠️ Component validation issues found:")
        for issue in issues:
            print(f"    {issue}")
    else:
        print("  ✅ Component validation passed")
    
    # Add necessary type imports if missing
    if 'import type { LucideIcon }' not in cleaned_content:
        import_statements = cleaned_content.split('\n', 1)
        cleaned_content = import_statements[0] + '\nimport type { LucideIcon } from "lucide-react";\n' + import_statements[1]
    
    # Save the file
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(cleaned_content)
    print(f"  ✓ Saved: {tsx_filename}")
    
    return issues

# Print confirmation
print("✅ Component validation and fixes functions loaded")

✅ Component validation and fixes functions loaded


In [11]:
# Cell 5 - Main Execution

def process_file(client, md_file, metadata):
    """
    Process a single file to generate its component.
    
    Args:
        client: Anthropic client instance
        md_file: Path object for the markdown file
        metadata: Dictionary containing file metadata
        
    Returns:
        tuple: (success status, any issues)
    """
    try:
        print(f"  ⌛ Creating prompt for: {metadata['title']}")
        prompt = create_prompt(metadata['title'], metadata['summary'])
        
        print("  ⌛ Generating and refining component...")
        component_code, issues = generate_component_with_refinement(
            client, 
            metadata['title'], 
            prompt
        )
        
        if component_code:
            save_tsx_file(component_code, md_file.name, OUTPUT_DIR)
            return True, issues
        else:
            print(f"  ❌ Failed to generate component for: {md_file.name}")
            return False, ["Generation failed"]
            
    except Exception as e:
        print(f"  ❌ Error processing file: {str(e)}")
        return False, [f"Processing error: {str(e)}"]

def main():
    """Main execution function for the component generator."""
    print("\n🚀 Starting AI Component Generator...\n")
    start_time_total = time.time()
    
    try:
        # Initialize Anthropic client
        client = anthropic.Client(api_key=os.getenv('ANTHROPIC_API_KEY'))
        
        # Check directories
        print("📂 Checking directories...")
        if not os.path.exists(INPUT_DIR):
            raise Exception(f"Input directory not found: {INPUT_DIR}")
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)
            print(f"  ✓ Created output directory: {OUTPUT_DIR}")
        
        # Get existing component names
        print("\n📂 Checking existing components...")
        existing_components = get_existing_component_names(OUTPUT_DIR)
        if existing_components:
            print(f"  ✓ Found {len(existing_components)} existing components")
        
        # Get list of all .md files that don't have corresponding .tsx files
        all_md_files = []
        for md_file in Path(INPUT_DIR).glob('*.md'):
            if md_file.stem not in existing_components:
                all_md_files.append(md_file)
        
        total_available = len(all_md_files)
        print(f"\n📁 Found {total_available} unprocessed files")
        
        if total_available == 0:
            print("❌ No new files to process")
            return
        
        # Select 3 random files (or all if less than 3 available)
        num_files = min(3, total_available)
        md_files = random.sample(all_md_files, num_files)
        
        print(f"\n🎲 Randomly selected {num_files} files to process")
        
        # Track statistics
        successful = 0
        failed = 0
        all_issues = []
        
        # Process each file
        for index, md_file in enumerate(md_files, 1):
            print(f"\n📝 Processing file {index}/{num_files}: {md_file.name}")
            start_time_file = time.time()
            
            try:
                print("  ⌛ Reading file...")
                with open(md_file, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                print("  ⌛ Extracting metadata...")
                metadata = extract_frontmatter(content)
                if not metadata:
                    print("  ❌ Could not extract metadata")
                    failed += 1
                    continue
                
                # Process the file
                success, issues = process_file(client, md_file, metadata)
                if success:
                    successful += 1
                    if issues:
                        all_issues.extend(issues)
                else:
                    failed += 1
                    all_issues.extend(issues)
                
                elapsed_time = time.time() - start_time_file
                print(f"  ⏱️ Time taken: {format_time(elapsed_time)}")
                
            except Exception as e:
                print(f"  ❌ Error: {str(e)}")
                failed += 1
        
        # Print summary
        total_time = time.time() - start_time_total
        print("\n====== Summary ======")
        print(f"✅ Successfully processed: {successful}")
        print(f"❌ Failed: {failed}")
        print(f"⏱️ Total time: {format_time(total_time)}")
        
        if all_issues:
            print("\n⚠️ Common issues found:")
            issue_count = {}
            for issue in all_issues:
                issue_count[issue] = issue_count.get(issue, 0) + 1
            for issue, count in sorted(issue_count.items(), key=lambda x: x[1], reverse=True):
                print(f"  • {issue} (×{count})")
        
    except Exception as e:
        print(f"\n❌ Fatal error: {str(e)}")
        raise
    
    print("\n✨ Process completed!")


In [12]:
# Cell 6
if __name__ == "__main__":
    main()


🚀 Starting AI Component Generator...

📂 Checking directories...

📂 Checking existing components...
  ✓ Found 8 existing components

📁 Found 812 unprocessed files

🎲 Randomly selected 3 files to process

📝 Processing file 1/3: bayesian-inference.md
  ⌛ Reading file...
  ⌛ Extracting metadata...
  ⌛ Creating prompt for: Bayesian Inference
  ⌛ Generating and refining component...

  ⌛ Stage 1: Generating initial component for Bayesian Inference...

=== System Prompt ===
        
You are a creative expert React developer and AI professor specializing in educational components for 15 to 18-year-old humans. 
Your components must strictly follow these technical requirements:

1. Architecture:
- "use client" directive at start (first line)
- import { useState, useEffect } from "react"; as the second line
- Only useState and useEffect hooks
- Only Lucide icons for visuals
- Only Tailwind CSS for styling
- No external libraries/components
- File extension: .tsx

2. TypeScript Implementation:
in